# Fine-tuning LLM to Generate Persian Product Catalogs in JSON Format

In this example, we will fine-tune an LLM to generae Persian product catalogs and produce structured output in JSON format.

The fine-tuned LLM is list on [their HuggingFace account](https://huggingface.co/BaSalam/Llama2-7b-entity-attr-v1).

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [ ]:
# base model to train
model_name = 'NousResearch/Llama-2-7b-chat-hf'
# instruction dataset to use
dataset_name = 'BaSalam/entity-attribute-dataset-GPT-3.5-generated-v1'
# fine-tuned lora adapter name
new_model = 'llama-persian-catalog-generator'

# LoRA parameters
lora_r = 64
lora_alpha = lora_r * 2
lora_dropout = 0.l
target_modules = ['q_proj', 'v_proj', 'k_proj']

# Additionally, Q-LoRA parameters
load_in_4bit= True
bnb_4bit_compute_dtype = 'float16'
bnb_4bit_quant_type = 'nf4'
bnb_4bit_use_double_quant = True

**LoRA** stores changes in weights by constructing and adding a low-rank matrix to each model layer. This method opens only these layers for fine-tuning, without changing the original model weights or requiring lengthy training. The resulting weights are lightweight and can be produced multiple times, allowing for the fine-tuning of multiple tasks with an LLM loaded into RAM.

**Q-LoRA** enables LLMs to run on smaller GPUs by using 4-bit quantization. This method preserves the full performance of 16-bit fine-tuning while reducing memory usage, making it possible to fine-tune models with up to 65B parameters on a single 48GB GPU. Q-LoRA combines 4-bit NormalFloat data types, double quantization, and paged optimizers to manage memory efficiently. It allows fine-tuning of models with low-rank adapters, significantly enhancing accessibilitiy for AI model development.

In [ ]:
# Training hyperparameters
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
learning_rate = 0.00015
weight_decay = 0.01
optim = 'paged_adamw_32bit'
lr_scheduler_type = 'cosine'
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 0
logging_steps = 25

# SFT parameters
max_seq_length = None
packing = False
device_map = {"": 0}

# Dataset parameters
use_special_template = True
response_template = " ### Answer:"
instruction_prompt_template = '"### Human:"'
use_llama_like_model = True

## Model training

In [ ]:
# load dataset
dataset = load_dataset(dataset_name, split="train")
percent_of_train_dataset = 0.95
other_columns = [
    i for i in dataset.column_names
    if i not in ['instruction', 'output']
]

dataset = dataset.remove_columns(other_columns)
split_dataset = dataset.train_test_split(
    train_size=int(dataset.num_rows * percent_of_train_dataset),
    seed=111,
    shuffle=False
)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']
print(f"Size of the train dataset: {len(train_dataset)}")
print(f"Size of the eval dataset: {len(eval_dataset)}")

In [ ]:
# load LoRA config
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    task_type='CAUSAL_LM',
    target_modules=target_modules
)

The parameters in the `LoraConfig`:
* `r` - rank of the low-rank matrices used in LoRA, controling the dimensionality of the low-rank adaptation and directly impacting the model's capacity to adapt and the computational cost.
* `lora_alpha` - controls the scaling factor for the low-rank adaptation matrices. Higher alpha increases the model's capacity to learn new tasks.
* `lora_dropout` - dropout rate for LoRA, preventing overfitting during fine-tuning.
* `bias` - specifies whether to add a bias term to the low-rank matrices. `'none'` means that no bias term will be added.
* `task_type` - defines the type of task for which the model is being fine-tuned.
* `tatget_modules` - specifies the modules in the model to which LoRA will be applied.

In [ ]:
# load Q-LoRA config
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant,
)

The **BitsAndBytes (bnb)** library provides efficient memory management and compression techniques for PyTorch models. It defines how the model weights will be loaded and quantized in 4-bit precision, which is useful for reducing memory usage and potentially speeding up inference.

The parameters in the `BitsAndBytesConfig` for the Q-LoRA setup:
* `load_in_4bit` - determines whether to load the model in 4-bit precision.
* `bnb_4bit_quant_type` - specifies the type of 4-bit quantization to use. Here we set to NF4 (NormalFloat 4) quantization type, which is information-theoretically optimal for normally distributed weights, providing an efficient way to quatnize the model for fine-tuning.
* `bnb_4bit_compute_dtype` - sets the data type used for computations involving the quantized model. In Q-LoRA, it is set to `"float16"`, which is commonly used for mixed-precision training to balance performance and precision.
* `bnb_4bit_use_double_quant` - determines whether to use dobule quantization.

Q-LoRA employs two distinct data types (`quant_type` and `compute_type`):
* one for storing base model weights (in here 4-bit NormalFloat), and
* another for computational operations (16-bit).

During the forward and backward passes, Q-LoRA dequantizes the weights from the storage format to the computational format. However, it only calculates gradients for the LoRA parameters, which utilize 16-bit bfloat. This approach ensures that weights are decompressed only when necessary, maintaining low memory usage throughout both training and inference phases.

In [ ]:
# load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False

In [ ]:
# training arguments
training_args = TrainingArguments(
    output_dir=new_model,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    grane_checkpointing=gradient_checkpointing,
    group_by_length=group_by_length
    lr_scheduler_type=lr_scheduler_type
)

In [ ]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right' # fix weird overflow issue with fp16 training
if not tokenizer.chat_template:
    tokenizer.chat_template = "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}"

The chat template is used to understand the structure of the conversation between the user and the model during model training. A series of reserved phrases are created to separate the user's message and the model's response. This ensures that the model precisely understands where each message comes from and maintains a sense of the conversational structure.

Adhering to a chat template helps increase accuracy in the intended task; however, when there is a distribution shift between the fine-tuning dataset and the model, using a specific chat template may be even more helpful.

In [ ]:
def special_formatting_prompts(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"{instruction_prompt_template}{example['instruction'][i]\n{response_template} {example['output'][i]}}"
        output_texts.append(text)
    return output_texts


def normal_formatting_prompts(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        chat_temp = [
            {'role': 'system', 'content': example['instruction'][i]},
            {'role': 'assistant', 'content': example['output'][i]}
        ]
        text = tokenizer.apply_chat_template(chat_temp, tokenize=False)
        output_texts.append(text)
    return output_texts

In [ ]:
if use_special_template:
    formatting_func = special_formatting_prompts
    if use_llama_like_model:
        response_template_ids = tokenizer.encode(response_template, add_special_tokens=False)[2:]
        collator = DataCollatorForCompletionOnlyLM(response_template=response_template_ids, tokenizer=tokenizer)
    else:
        collator = DataCollatorForCompletionOnlyLM(response_template=response_template, tokenizer=tokenizer)
else:
    formatting_func = normal_formatting_prompts

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    formatting_func=formatting_func,
    data_collator=collator,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
    packing=packing
)

The `SFTTrainer` is then instantiated to handle supervised fune-tuning (SFT) of the model. This trainer is specifically designed for SFT and includes additional parameters such as `formatting_func` and `packing` which are not found in standard trainers:
* `formatting_func` - a custom function to format training examples by combining instruction and response templates.
* `packing` - disables packing multiple samples into one sequence, which is not a standard parameter in the typical `Trainer` class.

In [ ]:
trianer.train()

# save fine-tuned lora
trainer.model.save_pretrained(new_model)

## Inference

In [ ]:
import torch
import gc

def clear_hardwares():
    torch.clear_autocast_cache()
    torch.cuda.ipc_collect()
    torch.cuda.empty_cache()
    gc.collect()

clear_hardwares()

In [ ]:
def generate(model, prompt: str, kwargs):
    tokenized_prompt = tokenizer(prompt, return_tensors='pt').to(model.device)
    prompt_length = len(tokenized_prompt.get('input_ids')[0])

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**tokenized_prompt, **kwargs) if kwargs else model.generate(**tokenized_prompt)
        output = tokenizer.decode(output_tokens[0][prompt_legnth:], skip_special_tokens=True)

    return output

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    new_model,
    return_dict=True,
    device_map='auto',
    token=''
)
tokenizer = AutoTokenizer.from_pretrained(new_model, max_length=max_seq_length)
model = PeftModel.from_pretrained(base_model, new_model)
del base_model

In [ ]:
sample = eval_dataset[0]
if use_special_template:
    prompt = f"{instruction_prompt_template}{sample['instruction']}\n{response_template}"
else:
    chat_temp = [{'role': 'system', 'content': sample['instruction']}]
    prompt = tokenizer.apply_chat_template(
        chat_temp,
        tokenize=False,
        add_generation_prompt=True
    )


gen_kwargs = {'max_new_tokens': 1024}
generated_texts = generate(model, prompt, gen_kwargs)
print(generated_texts)

## Merge to base model

In [ ]:
clear_hardwares()

In [ ]:
merged_model = model.merge_and_unload()
clear_hardwares()
del model

adapter_model_name = '<hf_account_name>/merged-llama-persian-catalog-generator'
merged_model.push_to_hub(adapter_model_name)

## Fast inference with `vllm`

In [ ]:
from vllm import LLM, SamplingParams

prompt = """### Question: here is a product title from a Iranian marketplace.  \n
    give me the Product Entity and Attributes of this product in Persian language.\n
    give the output in this json format: {'attributes': {'attribute_name' : <attribute value>, ...}, 'product_entity': '<product entity>'}.\n
    Don't make assumptions about what values to plug into json. Just give Json not a single word more.\n         \n
    product title:
"""
user_prompt_template = "### Question: "
response_template = " ### Answer:"

llm = LLM(
    model='BaSalam/Llama2-7b-entity-attr-v1',
    gpu_memory_utilization=0.9,
    trust_remote_code=True
)

product = 'مانتو اسپرت پانیذ قد جلوی کار حدودا 85 سانتی متر قد پشت کار حدودا 88 سانتی متر'
sampling_params = SamplingParams(temperature=0., max_tokens=75)
prompt = f"{user_prompt_template} {prompt}{product}\n {response_template}"
outputs = llm.generate(prompt, sampling_params)

print(outputs[0].outputs[0].text)